In [1]:
cd Documents/Thesis_aims/Data/

/home/temci/Documents/Thesis_aims/Data


In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import matplotlib.pyplot as plt

In [ ]:
##################### Loading Data ######################################################################

job_data = pd.read_table('Bons/total_jobs.csv')#, sep = r'\t', engine = 'python') #, nrows = 1000)
users_data = pd.read_table('Bons/total_users.csv')#, sep = '\t')
users_hist_data = pd.read_table('Bons/users_past_jobs.csv', sep = '\t')
apps_train_data = pd.read_table('Bons/applications_train.csv')#, sep = '\t')#[['WindowID'] == 1]
apps_test_data = pd.read_table('Bons/applications_test.csv')
interv1 = pd.read_table('Bons/positive_replies.csv', sep = '\t')

In [ ]:
%%time

########################## Removing HTML markups#########################################################

def clean_html(html):
    """
    Copied from NLTK package.
    Remove HTML markup from the given string.

    :param html: the HTML string to be cleaned
    :type html: str
    :rtype: str
    """

    # First we remove inline JavaScript/CSS:
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", "", str(html).strip())
    # Then we remove html comments. This has to be done before removing regular
    # tags since comments can contain '>' characters.
    cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", "", cleaned)
    # Next we can remove the remaining tags:
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
    # Finally, we deal with whitespace
    cleaned = re.sub(r"&nbsp;", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    cleaned = re.sub('\r', ' ', cleaned)
    cleaned = re.sub('/', ' ', cleaned)
    return cleaned.strip()

In [ ]:
%%time

############################### Preprecessing data ##################################################

user_defined_stop_words = ['r'] 
i = nltk.corpus.stopwords.words('english')
j = list(string.punctuation) + user_defined_stop_words
stopwords = set(i).union(j)
def preprocess(x):
    x = re.sub('<b>', ' ', str(x))
    x = re.sub('</b>', ' ', x.lower())
    x = re.sub('<p>', ' ', x.lower())
    x = re.sub('</p>', ' ', x.lower())
    x = re.sub('</span>', ' ', x.lower())
    x = re.sub('<span>', ' ', x.lower())
    x = re.sub('[^a-z\s]', ' ', x.lower())                  # get rid of noise
    x = re.sub(r'  ', ' ', x.lower())
    x = re.sub(r'   ', ' ', x.lower())
    x = [w for w in x.split() if w not in set(stopwords)]  # remove stopwords
    return ' '.join(x)                                     # join the list
corpus = job_data['Title'].append(users_hist_data['JobTitle'])
descriptions = corpus.apply(clean_html) 
descriptions = descriptions.apply(preprocess)#job_data['Description'].apply(preprocess)

In [ ]:
%%time

########################### the content-based recommender system ############################################

ds = descriptions.dropna()
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(ds.dropna())
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)


diction = {}
i = 0
for idx1, row in ds.iteritems():
    diction[i] = idx1
    i += 1


idx = 0
# for idx, row in ds.iteritems():
similar_all_items = pd.DataFrame(columns = ['JobID','UserID','Similarity'])
while idx < 16627 : #13084 :#16855 : 
    similar_items = pd.DataFrame()
#     similar_indices = cosine_similarities[idx][13084:].argsort()[:-21:-1]
    for i in cosine_similarities[idx][16627:].argsort()[:-21:-1]:#similar_indices :
        similar_items['JobID'] = [job_data.loc[diction[idx]].JobID]#:, 'JobID']
        similar_items['UserID'] = [users_hist_data.loc[diction[i+16627]].UserID]#:, 'UserID']
        similar_items['Similarity'] = [cosine_similarities[idx][i+16627]]
        similar_all_items = similar_all_items.append(similar_items, ignore_index = True)
#     similar_items = [(cosine_similarities[idx][i+13084], (dict[idx],dict[i+13084])) for i in similar_indices]
    idx += 1

In [ ]:
%%time

########################### Popularity-based, collaborative filtering and adding reciprocity ############################

reduc_users = np.unique(users_data['UserID'])
merge_all_job = pd.DataFrame()
merge_ = pd.DataFrame()
pop = pd.DataFrame()
popular = apps_train_data.groupby(['JobID'], as_index = False).count()\
    .rename(columns = {'UserID':'Count'}).sort_values(['Count'], ascending=False)[['JobID','Count']]

for user in reduc_users :
#     app_user = apps_data[apps_data['UserID'] == user ]
#     app_user = jobs[jobs['UserID'] == user]
    app_user = apps_train_data[apps_train_data['UserID'] == user]
    job_user = np.unique(app_user['JobID'])
#     sim_user = jobs[jobs['JobID'].isin(job_user)]
    popul = popular[~popular['JobID'].isin(job_user)][0:5]
    popul['UserID'] = user
    pop = pop.append(popul, ignore_index = True)
    
    sim_user = apps_train_data[apps_train_data['JobID'].isin(job_user)]
    sim_user = sim_user[sim_user['UserID'] != user]
    sim_user = sim_user.groupby(['UserID'], as_index = False).count()\
       .rename(columns = {'JobID':'Count'}).sort_values(['Count'], ascending=False)[['UserID', 'Count']][:10]
#     sim_unique_user = np.unique(sim_user['UserID'])
#     us_job = jobs[jobs['UserID'].isin(sim_unique_user)]
#     us_job = jobs[jobs['UserID'].isin(sim_user['UserID'])]
    us_job = apps_train_data[apps_train_data['UserID'].isin(sim_user['UserID'])]
    us_job = us_job[~us_job['JobID'].isin(job_user)]
    us_job = us_job.groupby(['JobID'],as_index=False).count().rename(columns = {'UserID':'Count'})\
          .sort_values(['Count'], ascending=False)[['JobID','Count']]
#     us_job['UserId'] = user
#     us_job = us_job.rename(columns = {'JobID':'JobId'})
    merge_all_job = merge_all_job.append(us_job[0:40], ignore_index = True)
    test = similar_all_items[similar_all_items['UserID'] == user]#.sort_values(['Similarity'], ascending=False)
    us_job1 = us_job.merge(test, on = 'JobID')[0:40]
    if not us_job1.empty :
        us_job1['UserID'] = us_job1['UserID_y']
        us_job1 = us_job1[['Count','JobID','Similarity','UserID']]
        merge_ = merge_.append(us_job1, ignore_index = True)
    else :
        us_job['Similarity'] = 0
        merge_ = merge_.append(us_job, ignore_index = True)

In [ ]:
%%time

######################## Recall of the users RS ############################################################ 

from __future__ import division
distinct = np.unique(merge_all_job['UserID'])
i = 0
evaluation = pd.DataFrame()
for user in distinct :
    testt = pd.DataFrame()
    user_chosen= merge_all_job[merge_all_job['UserID'] == user][['UserID','JobID']]
    vue = apps_test_data[apps_test_data['UserID'] == user][['JobID']]
    vu = user_chosen.merge(vue, on = 'JobID')
    if len(vue) != 0 and len(user_chosen) != 0:
        testt['Recall'] = [len(vu)/len(vue)]
        testt['UserID'] = [user]
        evaluation = evaluation.append(testt, ignore_index = True).sort_values(['Recall'], ascending = False)

In [ ]:
%%time

############################### Recall of the reciprocal RS #############################################

from __future__ import division
distinct = np.unique(merge_all_job['UserID'])
i = 0
evaluation1 = pd.DataFrame()
for user in distinct :
    testt = pd.DataFrame()
    user_chosen= merge_[merge_['UserID'] == user][['UserID','JobID']]
    vue = apps_test_data[apps_test_data['UserID'] == user][['JobID']]
    vu = user_chosen.merge(vue, on = 'JobID')
    if len(vue) != 0 and len(user_chosen) != 0:
        testt['Recall'] = [len(vu)/len(vue)]
        testt['UserID'] = [user]
        evaluation1 = evaluation1.append(testt, ignore_index = True).sort_values(['Recall'], ascending = False)


In [ ]:
%%time

################## Recall of the popularity-based RS #######################################################

from __future__ import division
distinct = np.unique(pop['UserID'])
i = 0
evaluation2 = pd.DataFrame()
for user in distinct :
    testt = pd.DataFrame()
    user_chosen= pop[pop['UserID'] == user][['UserID','JobID']]
    vue = apps_test_data[apps_test_data['UserID'] == user][['JobID']]
    vu = user_chosen.merge(vue, on = 'JobID')
    if len(vue) != 0 and len(user_chosen) != 0:
        testt['Recall'] = [len(vu)/len(vue)]
        testt['UserID'] = [user]
        evaluation2 = evaluation2.append(testt, ignore_index = True).sort_values(['Recall'], ascending = False)

In [ ]:
%%time

############################### Precision of the reciprocal RS precision #############################

from __future__ import division
distinct = np.unique(merge_['UserID'])
i = 0
evaluation3 = pd.DataFrame()
for user in distinct :
    testt = pd.DataFrame()
    user_chosen= merge_[merge_['UserID'] == user][['UserID','JobID']]
    vue = apps_test_data[apps_test_data['UserID'] == user][['JobID']]
    vu = user_chosen.merge(vue, on = 'JobID')
    interviewed = vu.merge(interv1, on = ['UserID','JobID'])
    total_interviewed = interv1[interv1['UserID'] == user]
    if len(vu)!=0 and len(total_interviewed)!= 0 :
        listee = [] 
        listee.append(len(interviewed)/len(vu))
        testt['Precision'] = listee 
        testt['UserID'] = [user]
        evaluation3 = evaluation3.append(testt, ignore_index = True).sort_values(['Recall'], ascending = False)

In [ ]:
%%time

############################# Precision of the users RS precision #############################

from __future__ import division
distinct = np.unique(merge_all_job['UserID'])
i = 0
evaluation4 = pd.DataFrame()
for user in distinct :
    testt = pd.DataFrame()
    user_chosen= merge_all_job[merge_all_job['UserID'] == user][['UserID','JobID']]
    vue = apps_test_data[apps_test_data['UserID'] == user][['JobID']]
    vu = user_chosen.merge(vue, on = 'JobID')
    interviewed = vu.merge(interv1, on = ['UserID','JobID'])
    total_interviewed = interv1[interv1['UserID'] == user]
    if len(vu)!=0 and len(total_interviewed)!= 0 :
        listee = [] 
        listee.append(len(interviewed)/len(vu))
        testt['Precision'] = listee 
        testt['UserID'] = [user]
        evaluation4 = evaluation4.append(testt, ignore_index = True).sort_values(['Recall'], ascending = False)

In [ ]:
######################### Mean of the recommendations precisions ################################

# for users RS
print(evaluation4['Precision'].mean())

#for reciprocal RS
print(evaluation3['Precision'].mean())



In [ ]:
######################### Mean of the recommendations recalls ################################

# for users RS
print(evaluation['Recall'].mean())

#for reciprocal RS
print(evaluation1['Recall'].mean())

# for popularity-based RS
print(evaluation2['Recall'].mean())

In [ ]:
############## Plotting to compare popularity-based,users and reciprocal RS ###################################


#The data
recall_10 = (0.4429, 0.4455, 0.4459, 0.4460, 0.4460)
precision_popular = (0.0324, 0.0270,0.0260, 0.0237,0.0216)
recall_popular = (0.0063, 0.0104, 0.0199, 0.0272,0.0331)
recall_simple = (0.2029, 0.3203, 0.4226, 0.4779,0.5190)
indices = [5,10,20,30,40]#,50,60]
#Calculate optimal width
width = np.min(np.diff(indices))/4


fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.bar(indices-width,recall_10,width,color='b',label='Reciprocal RS')
ax1.bar(indices,recall_popular,width,color='y',label='Popularity-based RS')
ax1.bar(indices+width,recall_simple,width,color='r',label='Popularity-based RS')
ax1.set_xlabel('Top-N')
ax1.set_ylabel('Recall')
ax1.legend()
plt.show()

In [ ]:
################# Plotting to compare users (traditional) and reciprocal RS #########################################


#The data
precision_10 = (0.2564, 0.2612, 0.2615, 0.2615, 0.2615)
precision_simple = (0.0230, 0.0219, 0.0205, 0.0202, 0.0204)
indices = [5,10,20,30,40]
#Calculate optimal width
width = np.min(np.diff(indices))/3

fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(indices-width,precision_10,width,color='b',label='Reciprocal RS')
ax.bar(indices,precision_simple,width,color='r',label='User RS')
ax.set_xlabel('Top-N')
ax.set_ylabel('Precision')
ax.legend()
plt.show()